In [1]:
from sklearn import tree
import pandas as pd
import os
import numpy as np
df=pd.read_csv("Resources/Adults_Diabetes_NHANES_2011_2012.csv")

df.head()

,seqn,first,last,gender,age,race,education,marital,income,household_size,...,trigs,wbc,hgb,hct,platelets,s_cotinine,a1c,hdl,grip_strength,fev1_fvc_ratio
0,69220,Gwendolyn,Runolfsson,1,21,7,2.0,6.0,1.0,2,...,54.0,6.0,12.7,36.1,157.0,0.654,5.0,47.0,50.3,0.78
1,63030,Augustus,Farrell,0,21,1,2.0,6.0,3.0,4,...,83.0,6.9,15.1,44.4,226.0,0.221,5.2,40.0,90.1,0.84
2,64051,Aaron,Schmeler,0,21,2,3.0,5.0,4.0,3,...,256.0,8.2,14.4,41.3,266.0,0.011,5.1,38.0,72.7,0.83
3,65141,Bob,Bechtelar,0,21,1,2.0,5.0,4.0,4,...,57.0,6.6,14.7,43.0,206.0,16.300,5.1,55.0,86.6,0.83
4,64632,Hayden,Brekke,0,21,2,3.0,5.0,10.0,2,...,70.0,7.8,15.6,45.1,306.0,212.000,6.0,39.0,94.4,0.83


In [2]:
df_1=df.drop(columns=['first', 'last','drinks_day', 'depression','fev1_fvc_ratio'])
#Remember to Add back SEQN for ETL

df_1.head()



,seqn,gender,age,race,education,marital,income,household_size,insurance,gen_health,...,glob,trigs,wbc,hgb,hct,platelets,s_cotinine,a1c,hdl,grip_strength
0,69220,1,21,7,2.0,6.0,1.0,2,1,3.0,...,2.9,54.0,6.0,12.7,36.1,157.0,0.654,5.0,47.0,50.3
1,63030,0,21,1,2.0,6.0,3.0,4,2,3.0,...,2.8,83.0,6.9,15.1,44.4,226.0,0.221,5.2,40.0,90.1
2,64051,0,21,2,3.0,5.0,4.0,3,1,3.0,...,3.0,256.0,8.2,14.4,41.3,266.0,0.011,5.1,38.0,72.7
3,65141,0,21,1,2.0,5.0,4.0,4,1,4.0,...,2.6,57.0,6.6,14.7,43.0,206.0,16.300,5.1,55.0,86.6
4,64632,0,21,2,3.0,5.0,10.0,2,1,2.0,...,2.9,70.0,7.8,15.6,45.1,306.0,212.000,6.0,39.0,94.4


In [3]:
len(df_1)

5206

In [4]:
df1=df_1.dropna()
len(df1)

3260

In [5]:
#BMI 
#0= BMI less than 25
#1= BMI less than 30 and BMI great than 24
#2=BMI great than 30

conditions = [
   df1['bmi'] <25,
    (df1['bmi'] >=25) & (df1['bmi'] <30),
   df1['bmi'] >=30
]
choices = [0,1, 2]
df1['bmi_group'] = np.select(conditions, choices, default=1)

#df1['bmi_group'] = 1
#df1.loc[df1['bmi'] >=30,'bmi_group'] = 2
#df1.loc[df1['bmi'] <24,'bmi_group']= 0

df1[["bmi","bmi_group"]].sample(50)

/Users/tajudeenadeyemi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,bmi,bmi_group
4959,27.3,1
2044,33.5,2
1154,33.1,2
2980,33.2,2
2161,40.5,2
2072,29.9,1
3261,40.2,2
4337,45.2,2
876,27.3,1
3224,43.8,2


In [6]:


#Age Group 

# 0=  0-20
# 1=  21-30
# 2=  31-45
# 3=  46-65
# 4=  65+
conditions1 = [
    (df1['age'] >20)&(df1['age'] <31),
    (df1['age'] >30)&(df1['age'] <46),
    (df1['age'] >45)&(df1['age'] <66),
    df1['age'] >65
]
choices1 = [1, 2, 3,4]
df1['age_group'] = np.select(conditions1, choices1, default=0)





#df1['age_group'] = 0
#df1.loc[(df1['age'] >20)&(df1['age'] <31),'age_group'] = 1
#df1.loc[(df1['age'] >30)&(df1['age'] <46),'age_group']= 2
#df1.loc[(df1['age'] >45)&(df1['age'] <66),'age_group'] = 3
#df1.loc[df1['age'] >65,'age_group'] = 4

df1[["age","age_group"]].sample(50)

/Users/tajudeenadeyemi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,age,age_group
2436,47,3
1311,35,2
3907,63,3
1272,34,2
2918,52,3
2995,53,3
3358,57,3
4559,72,4
549,26,1
4793,78,4


In [7]:
#Blood Pressure Group 
#Systolic Blood Pressure less than 120 and Diastolic Blood Pressure of less than 80
#Systolic Blood Pressure between 120 and 129 and Diastolic Blood Pressure of less than 80
#Systolic Blood Pressure greater than 130 or  Diastolic Blood Pressure of greater than 80

conditions2 = [
    (df1['sys_bp'] <120)&(df1['dia_bp'] <80),
   (df1['sys_bp'] >119)&(df1['sys_bp'] <=129)&(df1['dia_bp'] <80),
    (df1['sys_bp'] >129)|(df1['dia_bp'] >=80),
 
]
choices2 = [0, 1, 2]
df1['bp_group'] = np.select(conditions2, choices2, default=0)
#df1['bp_group'] = 0
#df1.loc[(df['sys_bp'] <120)&(df1['dia_bp'] <80),'bp_group'] = 0
#df1.loc[(df['sys_bp'] >119)&(df1['sys_bp'] <=129)&(df['dia_bp'] <80),'bp_group'] = 1
#df1.loc[(df['sys_bp'] >129)|(df1['dia_bp'] >=80),'bp_group'] = 2


df1[["sys_bp","dia_bp","bp_group"]].sample(50)


/Users/tajudeenadeyemi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,sys_bp,dia_bp,bp_group
2984,114.0,62.0,0
1099,114.0,66.0,0
3485,126.0,82.0,2
4535,122.0,62.0,1
225,130.0,86.0,2
570,152.0,98.0,2
4726,138.0,68.0,2
3479,112.0,74.0,0
1968,116.0,64.0,0
3756,136.0,80.0,2


In [8]:
df2=df1.drop(columns=['seqn'])

In [9]:
target = df2["diabetes"] 
target_names = [1,2,3]

In [10]:
data = df2.drop("diabetes", axis=1) 
feature_names = data.columns 
data.head()

,gender,age,race,education,marital,income,household_size,insurance,gen_health,asthma,...,hgb,hct,platelets,s_cotinine,a1c,hdl,grip_strength,bmi_group,age_group,bp_group
0,1,21,7,2.0,6.0,1.0,2,1,3.0,2.0,...,12.7,36.1,157.0,0.654,5.0,47.0,50.3,0,1,0
1,0,21,1,2.0,6.0,3.0,4,2,3.0,1.0,...,15.1,44.4,226.0,0.221,5.2,40.0,90.1,1,1,0
2,0,21,2,3.0,5.0,4.0,3,1,3.0,2.0,...,14.4,41.3,266.0,0.011,5.1,38.0,72.7,1,1,0
3,0,21,1,2.0,5.0,4.0,4,1,4.0,2.0,...,14.7,43.0,206.0,16.300,5.1,55.0,86.6,0,1,0
4,0,21,2,3.0,5.0,10.0,2,1,2.0,2.0,...,15.6,45.1,306.0,212.000,6.0,39.0,94.4,2,1,2


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42, shuffle=True)

In [12]:
clf = tree.DecisionTreeClassifier() 
clf = clf.fit(X_train, y_train) 
clf.score(X_test, y_test)

0.8564417177914111

In [13]:
from sklearn.ensemble import RandomForestClassifier 
rf = RandomForestClassifier(n_estimators=75) 
rf = rf.fit(X_train, y_train) 
rf.score(X_test, y_test)

0.9141104294478528

In [14]:
feature_imp=sorted(zip(rf.feature_importances_, feature_names), reverse=True)

feature_imp[1][0].dtype

dtype('float64')

In [15]:
remlist=['age', 'bmi','height_cm','waist_cm','weight_kg','dia_bp','sys_bp']
feats= []

for x in feature_imp: 
    if x[0]>0.01 and x[1] not in remlist:
        feats.append(x[1])
        


feats        





['a1c',
 'glucose',
 'alb_cr_ratio',
 't_chol',
 'ldh',
 'trigs',
 'cr',
 'potassium',
 'platelets',
 'grip_strength',
 'alk_phos',
 'hdl',
 'ast',
 'chloride',
 'wbc',
 'hct',
 'alt',
 'cpk',
 'u_acid',
 'iron',
 'hgb',
 'bun',
 'ca',
 'phos',
 'glob',
 'sodium',
 'hypertension',
 's_cotinine',
 't_protein',
 'alb',
 'gen_health',
 't_bilirubin']

In [16]:
group_list=["bmi_group", "age_group", "bp_group"]


for x in group_list: 
    if x not in feats: 
        feats.append(x)
        
feats

['a1c',
 'glucose',
 'alb_cr_ratio',
 't_chol',
 'ldh',
 'trigs',
 'cr',
 'potassium',
 'platelets',
 'grip_strength',
 'alk_phos',
 'hdl',
 'ast',
 'chloride',
 'wbc',
 'hct',
 'alt',
 'cpk',
 'u_acid',
 'iron',
 'hgb',
 'bun',
 'ca',
 'phos',
 'glob',
 'sodium',
 'hypertension',
 's_cotinine',
 't_protein',
 'alb',
 'gen_health',
 't_bilirubin',
 'bmi_group',
 'age_group',
 'bp_group']

In [17]:
diab_df =df2[feats]

In [18]:
diab_df

,a1c,glucose,alb_cr_ratio,t_chol,ldh,trigs,cr,potassium,platelets,grip_strength,...,sodium,hypertension,s_cotinine,t_protein,alb,gen_health,t_bilirubin,bmi_group,age_group,bp_group
0,5.0,82.0,11.77,118.0,75.0,54.0,0.44,3.4,157.0,50.3,...,138.0,2.0,0.654,7.3,4.4,3.0,0.8,0,1,0
1,5.2,81.0,2.37,172.0,137.0,83.0,0.81,3.5,226.0,90.1,...,137.0,2.0,0.221,7.3,4.5,3.0,1.2,1,1,0
2,5.1,87.0,3.73,168.0,112.0,256.0,0.82,3.5,266.0,72.7,...,138.0,2.0,0.011,7.5,4.5,3.0,0.4,1,1,0
3,5.1,91.0,3.74,144.0,87.0,57.0,0.73,3.6,206.0,86.6,...,136.0,2.0,16.300,7.4,4.8,4.0,0.9,0,1,0
4,6.0,89.0,3.13,104.0,104.0,70.0,1.07,4.7,306.0,94.4,...,142.0,2.0,212.000,7.2,4.3,2.0,0.8,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5196,5.8,98.0,49.63,185.0,127.0,80.0,0.55,3.7,178.0,47.2,...,140.0,1.0,0.011,7.2,4.4,3.0,0.9,0,4,2
5197,6.0,100.0,9.40,166.0,163.0,105.0,1.15,3.7,189.0,75.6,...,143.0,2.0,0.011,8.0,4.2,3.0,1.0,1,4,2
5199,7.0,175.0,187.41,176.0,155.0,104.0,0.92,4.7,273.0,33.1,...,137.0,1.0,0.011,6.9,3.8,3.0,0.5,2,4,2
5201,6.5,126.0,11.43,171.0,119.0,130.0,0.73,3.9,205.0,65.2,...,138.0,2.0,0.269,6.9,4.2,2.0,0.7,2,4,2


In [19]:
data = df2.drop("diabetes", axis=1) 
feature_names = data.columns 
data.head()

,gender,age,race,education,marital,income,household_size,insurance,gen_health,asthma,...,hgb,hct,platelets,s_cotinine,a1c,hdl,grip_strength,bmi_group,age_group,bp_group
0,1,21,7,2.0,6.0,1.0,2,1,3.0,2.0,...,12.7,36.1,157.0,0.654,5.0,47.0,50.3,0,1,0
1,0,21,1,2.0,6.0,3.0,4,2,3.0,1.0,...,15.1,44.4,226.0,0.221,5.2,40.0,90.1,1,1,0
2,0,21,2,3.0,5.0,4.0,3,1,3.0,2.0,...,14.4,41.3,266.0,0.011,5.1,38.0,72.7,1,1,0
3,0,21,1,2.0,5.0,4.0,4,1,4.0,2.0,...,14.7,43.0,206.0,16.300,5.1,55.0,86.6,0,1,0
4,0,21,2,3.0,5.0,10.0,2,1,2.0,2.0,...,15.6,45.1,306.0,212.000,6.0,39.0,94.4,2,1,2


In [20]:
target = df2["diabetes"] 
target_names = [1,2,3]

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42, shuffle=True)

In [22]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
from keras.utils import to_categorical 

y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

ModuleNotFoundError: No module named 'keras'